In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../raw_data/merged_df.csv')

In [3]:
df.head()

,state,year_season,sightings,avg_duration(seconds),sightings_cities,shape,sightings_days,year_x,season_x,population,year_y,season_y
0,mo,1975-autumn,1.0,60.0,1.0,disk,1.0,1975,autumn,4808.308,1975.0,autumn
1,mo,1975-summer,2.0,690.0,2.0,circle,2.0,1975,summer,4808.308,1975.0,summer
2,mo,1976-summer,4.0,44115.0,4.0,light,2.0,1976,summer,4839.029,1976.0,summer
3,mo,1977-autumn,1.0,180.0,1.0,unknown,1.0,1977,autumn,4863.173,1977.0,autumn
4,mo,1977-spring,2.0,75.0,2.0,circle,2.0,1977,spring,4863.173,1977.0,spring


In [4]:
def turn_season_to_number(string):
    if string == 'winter':
        return 1
    if string == 'spring':
        return 2
    if string == 'summer':
        return 3
    else:
        return 4

season_numbers_df = pd.DataFrame({'season_number':list(map(turn_season_to_number, list(df['season_y'])))})  


In [5]:
df = pd.concat([df,season_numbers_df], axis =1)

In [6]:
df.sort_values(by = ['state','year_y','season_number'], ascending = True, inplace =True)
df.drop(columns= ['year_x','season_x'], inplace = True)

In [7]:
df['sightings'] = df['sightings'].replace(np.nan, 0)
df['sightings_days'] = df['sightings_days'].replace(np.nan, 0)
df['avg_duration(seconds)'] = df['avg_duration(seconds)'].replace(np.nan, 0)
df['sightings_cities'] = df['sightings_cities'].replace(np.nan, 0)
df['shape'].fillna('No_shape', inplace=True)

In [8]:
aux = df.loc[df.state == 'ak', 'sightings_days'].shift(periods = 1).rename('sightings_t+1')
pd.concat([df.loc[df.state == 'ak'],aux], axis =1)

,state,year_season,sightings,avg_duration(seconds),sightings_cities,shape,sightings_days,population,year_y,season_y,season_number,sightings_t+1
1125,ak,1975-summer,1.0,2700.0,1.0,light,1.0,376.170,1975.0,summer,3,NaN
1126,ak,1976-autumn,1.0,7200.0,1.0,sphere,1.0,400.969,1976.0,autumn,4,1.0
1127,ak,1978-summer,1.0,180.0,1.0,rectangle,1.0,404.766,1978.0,summer,3,1.0
1128,ak,1980-summer,1.0,1500.0,1.0,rectangle,1.0,405.315,1980.0,summer,3,1.0
1129,ak,1982-summer,1.0,900.0,1.0,sphere,1.0,449.606,1982.0,summer,3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1239,ak,2020-winter,4.0,300.0,4.0,Flash,4.0,731.158,2020.0,winter,1,6.0
1237,ak,2020-spring,4.0,1080.0,4.0,Changing,4.0,731.158,2020.0,spring,2,4.0
1238,ak,2020-summer,2.0,1380.0,2.0,Triangle,2.0,731.158,2020.0,summer,3,4.0
1236,ak,2020-autumn,6.0,860.0,5.0,Light,5.0,731.158,2020.0,autumn,4,2.0


In [9]:
def set_time_series(df):
    
    states_ = list(df.state.unique())
    df_aux = pd.DataFrame()
    
    for st_ in states_:
        aux = df.loc[df.state == st_, 'sightings_days'].shift(periods = 1).rename('sightings_t+1')
        df_aux = df_aux.append(pd.concat([df.loc[df.state == st_],aux], axis =1))
    return df_aux 

In [10]:
df = set_time_series(df)
df['sightings_t+1'] = df['sightings_t+1'].replace(np.nan, 0)
df.dropna(axis = 0, inplace = True)

In [11]:
df

,state,year_season,sightings,avg_duration(seconds),sightings_cities,shape,sightings_days,population,year_y,season_y,season_number,sightings_t+1
1125,ak,1975-summer,1.0,2700.0,1.0,light,1.0,376.170,1975.0,summer,3,0.0
1126,ak,1976-autumn,1.0,7200.0,1.0,sphere,1.0,400.969,1976.0,autumn,4,1.0
1127,ak,1978-summer,1.0,180.0,1.0,rectangle,1.0,404.766,1978.0,summer,3,1.0
1128,ak,1980-summer,1.0,1500.0,1.0,rectangle,1.0,405.315,1980.0,summer,3,1.0
1129,ak,1982-summer,1.0,900.0,1.0,sphere,1.0,449.606,1982.0,summer,3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5489,wy,2019-autumn,4.0,285.0,4.0,Circle,4.0,580.116,2019.0,autumn,4,4.0
5496,wy,2020-winter,4.0,210.0,3.0,Formation,3.0,582.328,2020.0,winter,1,4.0
5494,wy,2020-spring,8.0,1170.0,8.0,Unknown,7.0,582.328,2020.0,spring,2,3.0
5495,wy,2020-summer,5.0,1152.0,5.0,Diamond,5.0,582.328,2020.0,summer,3,7.0


In [13]:
df.shape

(6514, 12)

In [12]:
df.to_csv('../../raw_data/final_df.csv',header=True,index=False)